In [ ]:
import time
import json
import csv
from urllib.parse import urlparse, parse_qs

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager

# --- 1. 설정 변수 ---
START_PAGE = 1
END_PAGE = 100
BASE_URL = "https://www.aladin.co.kr/shop/wbrowse.aspx?BrowseTarget=List&ViewRowsCount=25&ViewType=Detail&PublishMonth=0&SortOrder=4&page={}&Stockstatus=1&PublishDay=84&CID=336&CustReviewRankStart=&CustReviewRankEnd=&CustReviewCountStart=&CustReviewCountEnd=&PriceFilterMin=&PriceFilterMax=&SearchOption="
# 파일명 설정
BACKUP_FILENAME_FORMAT = "aladin_reviews_backup_page_{}.csv"
FINAL_FILENAME = "aladin_reviews_completed.csv"

def create_driver():
    """웹 드라이버를 생성하고 반환합니다."""
    options = webdriver.ChromeOptions()
    # options.add_argument('--headless')  # 필요 시 백그라운드 실행
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36")
    
    try:
        service = Service(ChromeDriverManager().install())
        driver = webdriver.Chrome(service=service, options=options)
        print("Chrome 드라이버가 성공적으로 생성되었습니다.")
        return driver
    except Exception as e:
        print(f"오류: Chrome 드라이버 생성에 실패했습니다: {e}")
        return None

def extract_book_metadata(driver):
    """
    페이지의 JSON-LD 스크립트에서 책의 메타데이터를 추출합니다.
    구조화된 데이터를 사용하므로 HTML 구조 변경에 더 안정적입니다.
    """
    try:
        scripts = driver.find_elements(By.CSS_SELECTOR, 'script[type="application/ld+json"]')
        for script in scripts:
            data = json.loads(script.get_attribute('innerHTML'))
            
            # JSON-LD 데이터 구조에 따라 책 정보를 찾음
            target_data = None
            if data.get('@type') == 'Book':
                target_data = data
            elif data.get('@type') == 'BreadcrumbList':
                for item in data.get('itemListElement', []):
                    if item.get('item', {}).get('@type') == 'Book':
                        target_data = item.get('item')
                        break
            
            if target_data:
                author_info = target_data.get('author', {})
                # 저자가 여러 명일 경우를 대비하여 리스트인지 확인
                if isinstance(author_info, list) and author_info:
                    author_name = author_info[0].get('name', '저자 정보 없음')
                else:
                    author_name = author_info.get('name', '저자 정보 없음')

                return {
                    'title': target_data.get('name', '제목 없음'),
                    'author': author_name,
                    'publisher': target_data.get('publisher', {}).get('name', '출판사 없음'),
                    'genre': target_data.get('genre', '장르 없음'),
                    'rating': target_data.get('aggregateRating', {}).get('ratingValue', 0),
                    'review_count': target_data.get('aggregateRating', {}).get('reviewCount', 0)
                }
    except json.JSONDecodeError:
        print("  - 주의: JSON-LD 데이터 파싱에 실패했습니다.")
    except Exception as e:
        print(f"  - 오류: 책 메타데이터 추출 중 오류 발생: {e}")
    return None

def collect_reviews(driver):
    """
    Selenium을 사용하여 현재 페이지의 모든 리뷰를 수집합니다.
    '더보기' 버튼을 끝까지 클릭하며 동적으로 로드되는 리뷰를 가져옵니다.
    """
    collected_reviews, processed_texts = [], set()
    
    # 페이지 하단으로 스크롤하여 리뷰 영역이 로드되도록 유도
    try:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight * 0.7);")
        time.sleep(1)
    except Exception as e:
        print(f"  - 주의: 스크롤 실행 중 오류 발생: {e}")

    while True:
        # 현재 페이지 소스를 파싱하여 리뷰 추출
        soup = BeautifulSoup(driver.page_source, "html.parser")
        review_elements = soup.select("div.hundred_list")
        
        new_reviews_found = False
        for review_elem in review_elements:
            text_span = review_elem.select_one('span[id^="spnPaper"]')
            text = text_span.get_text(strip=True) if text_span else ""
            
            # 중복되지 않은 새로운 리뷰만 추가
            if text and text not in processed_texts:
                star_images = review_elem.select('div.HL_star img[src*="icon_star_on.png"]')
                rating = len(star_images)
                
                collected_reviews.append({"rating": rating, "text": text})
                processed_texts.add(text)
                new_reviews_found = True
        
        # '더보기' 버튼 클릭 시도
        try:
            wait = WebDriverWait(driver, 5)
            more_button_xpath = '//*[@id="divReviewPageMore"]/div[1]/a'
            more_button = wait.until(EC.element_to_be_clickable((By.XPATH, more_button_xpath)))
            
            # JavaScript 클릭이 더 안정적일 수 있음
            driver.execute_script("arguments[0].click();", more_button)
            time.sleep(1.5) # 컨텐츠 로딩 대기
        except TimeoutException:
            # '더보기' 버튼이 더 이상 없으면 루프 종료
            break
        except Exception as e:
            print(f"  - 주의: '더보기' 버튼 클릭 중 오류 발생: {e}")
            break
            
    return collected_reviews

def save_to_csv(data, filename, is_final=False):
    """수집된 데이터를 CSV 파일로 저장합니다."""
    if not data:
        if is_final:
            print("수집된 데이터가 없어 최종 파일을 저장하지 않습니다.")
        return

    try:
        print(f"\n파일 저장 중... ({filename})")
        with open(filename, "w", newline="", encoding="utf-8-sig") as csvfile:
            # 데이터가 있는 경우에만 헤더를 결정
            fieldnames = data[0].keys()
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()
            writer.writerows(data)
        print(f"저장 완료: 총 {len(data)}개의 리뷰가 {filename}에 저장되었습니다.")
    except Exception as e:
        print(f"오류: 파일 저장 실패: {e}")

# --- 2. 메인 실행 로직 ---
def main():
    print("알라딘 도서 리뷰 크롤러를 시작합니다.")
    print(f"수집 범위: 페이지 {START_PAGE}부터 {END_PAGE}까지")

    driver = create_driver()
    if not driver:
        print("프로그램을 종료합니다.")
        return

    collected_reviews = []
    total_books_processed = 0

    for page_num in range(START_PAGE, END_PAGE + 1):
        print(f"\n--- 목록 페이지 {page_num}/{END_PAGE} 처리 시작 ---")
        try:
            driver.get(BASE_URL.format(page_num))
            time.sleep(2) # 페이지 로딩 대기

            # 책 상세 페이지로 연결되는 링크 수집
            book_elements = driver.find_elements(By.CSS_SELECTOR, 'a.bo3')
            book_links = [elem.get_attribute('href') for elem in book_elements if elem.get_attribute('href')]

            if not book_links:
                print(f"경고: 페이지 {page_num}에서 책 링크를 찾을 수 없습니다.")
                continue
            
            print(f"페이지 {page_num}에서 {len(book_links)}개의 책을 발견했습니다.")

            for i, book_url in enumerate(book_links, 1):
                try:
                    # URL에서 ItemId 추출 (더 안정적인 방법)
                    parsed_url = urlparse(book_url)
                    item_id = parse_qs(parsed_url.query).get('ItemId', [None])[0]
                    if not item_id:
                        print(f"  [{i}/{len(book_links)}] ItemId를 찾을 수 없어 건너뜁니다.")
                        continue
                        
                    print(f"\n  [{i}/{len(book_links)}] 책 처리 중 (ItemId: {item_id})")
                    
                    driver.get(book_url)
                    time.sleep(2)

                    metadata = extract_book_metadata(driver)
                    if not metadata:
                        print("  - 메타데이터 수집에 실패하여 이 책을 건너뜁니다.")
                        continue
                    
                    print(f"  - 제목: {metadata['title'][:40]}...")

                    reviews = collect_reviews(driver)
                    if not reviews:
                        print("  - 수집된 리뷰가 없습니다.")
                    else:
                        print(f"  - {len(reviews)}개의 리뷰를 수집했습니다.")
                        for review in reviews:
                            review_id_counter = len(collected_reviews) + 1
                            processed_review = {
                                "review_id": f"aladin_{item_id}_{review_id_counter}",
                                "book_title": metadata['title'],
                                "book_author": metadata['author'],
                                "book_publisher": metadata['publisher'],
                                "book_category": metadata['genre'],
                                "review_text": review['text'],
                                "review_rating": review['rating'],
                                "book_avg_rating": metadata['rating'],
                                "source_website": "알라딘"
                            }
                            collected_reviews.append(processed_review)
                    
                    total_books_processed += 1

                except Exception as e:
                    print(f"  - 오류: 개별 책 처리 중 예외 발생: {e}")
                    continue
        
        except Exception as e:
            print(f"오류: 목록 페이지 {page_num} 처리 중 예외 발생: {e}")
            continue

        # 10 페이지마다 또는 마지막 페이지에서 중간 저장
        if (page_num % 10 == 0 or page_num == END_PAGE):
            backup_filename = BACKUP_FILENAME_FORMAT.format(page_num)
            save_to_csv(collected_reviews, backup_filename)

    driver.quit()
    print("\n브라우저를 종료했습니다.")
    
    # 최종 결과 저장
    save_to_csv(collected_reviews, FINAL_FILENAME, is_final=True)

    print("\n--- 최종 결과 ---")
    print(f"총 처리된 책: {total_books_processed}권")
    print(f"총 수집된 리뷰: {len(collected_reviews)}개")
    print("모든 작업을 완료했습니다.")


if __name__ == "__main__":
    main()